#CODE FOR THE CHARACTERIZATION OF THE SOLENOIDS MAGNETIC FIELD

This code provides the pieces of code needed to fin the center of a conducting cylinder, and make measurements along three different trajectories. A cylindric mapping, a mapping along a line, and cross-like measurements in different planes along the longitudinal axis.

# Libraries and functions

In [1]:
# Function definitions:

#cylinder_generator: Generates an array with a mesh of a cilinder given the dimensions (diameter and longitude) 
#                  and the distance between points

#vector_extraction: Transformation of an especific string to a float vector ('[PRB:18.6,0.0,-20.0:1]'->[18.6,0.0,-20.0])

#number_of_points: Returns the number of points in the cylinder mesh

import serial 
import time
import numpy as np
import math
def cylinder_generator (diameter, yvector, precision):

    Cylinder = [] #a List wich includes the diferent sections of the cylinder, each element has a different y coordinate
    Section_square = [] #a List containing all points in xz plane for a given y coordinate
    NumberOfPoints=round(diameter/precision)+1
   
    y = 0
    for h in yvector:
        Section_square = []
        y = h
        x,z = -diameter/2, -diameter/2
        for i in range(NumberOfPoints):
            for j in range (NumberOfPoints):
                if i%2==1:
                    Section_square.append([-x,y,z])
                else:
                    Section_square.append([x,y,z])
                x += precision
            z += precision
            x =- diameter/2
        Section_circle=[]
        for vector in Section_square:
            if (vector[0]**2+vector[2]**2) <= (diameter/2)**2:
                Section_circle.append(vector)
                
        Cylinder.append(Section_circle)
        
    return Cylinder


def vector_extraction(string):
    #de una string del tipo '[PRB:18.639,0.000,-20.000:1]' al vector [18.639,0.000,-20.000]
    string = string.strip('[]PRB:')
    string = string.rstrip('1')
    string =string.strip(':')
    lista_str = string.split(sep=',')
    lista_float = [float(x) for x in lista_str]
    return lista_float

def number_of_points(cylinder):
    return len(cylinder)*len(cylinder[0])

# Manual G-code sender
This piece of code is included so the user can send whatever piece of G-code to perform individual movements or operations with the XYZ positioner. 

In [12]:
grbl = serial.Serial('COM4',115200) #Open serial port for the CNC
grbl.write('\r\n\r\n'.encode('utf-8'))
time.sleep(2)   # Wait for grbl to initialize
grbl.flushInput()  # Flush startup text in serial input
grbl.write('$x\n'.encode('utf-8'))
grbl_out = grbl.readline() # Wait for grbl response with carriage return
print( ' : ' + grbl_out.strip().decode('utf-8'))

#Put here the code you want to send
grbl.write('g91\n'.encode('utf-8'))
out = grbl.readline()
out = grbl.readline()
print(out.decode('utf-8'))
grbl.write('g21\n'.encode('utf-8'))
out = grbl.readline()

grbl.write('g0 z2\n'.encode('utf-8'))
out = grbl.readline()

grbl.write('g04 P1\n'.encode('utf-8'))
time.sleep(2)

#Here finishes the code you want to send

a=input("Press any key when finished: ")
grbl.close()


 : [MSG:Caution: Unlocked]
ok

Press any key when finished: 


# Probing
This piece of code finds the center of a conductor cylinder. Before executing make sure that the conductor is properly connected to the GND signal of the OpenBuilds Blackbox, and that the probe is also correctly connected to SIG.
This part of the process is the most conflictive, Grbl does not seem to always send the same code, and this code is not robust against those changes. Here is a list of the errors and alarms that could pop up: http://domoticx.com/cnc-machine-grbl-error-list/ 

In [44]:
leftside = 0
rightside = 0
topside = 0
bottomside = 0
approxCircleDia = 100
probeFeed = 200

grbl = serial.Serial('COM4',115200)
grbl.write('\r\n\r\n'.encode('utf-8'))
time.sleep(2)
grbl.flushInput()
grbl.write('$x\n'.encode('utf-8'))
grbl_out = grbl.readline()
print('alarm quit: ', grbl_out.decode('utf-8'))
grbl.write('g21\n'.encode('utf-8'))
grbl_out = grbl.readline()
grbl.write('g91\n'.encode('utf-8'))
grbl_out = grbl.readline()
grbl.write('g01 z0.1 f100\n'.encode('utf-8'))
grbl_out = grbl.readline()
grbl.write('g10 p1 l20 x0 y0 z0\n'.encode('utf-8'))
grbl_out = grbl.readline()

#a=input()

#BUSCAMOS EL CENTRO EN X

grbl.write(('g38.2 x'+ str(approxCircleDia/2) + ' f' + str(probeFeed)+'\n').encode('utf-8'))
grbl_out = grbl.readline()
print('right probe: ',grbl_out.decode('utf-8'))
grbl_out = grbl.readline()
grbl_out = grbl_out.strip().decode('utf-8') 
print('position: ', grbl_out)
if grbl_out[0:4] == '[PRB':  
    vector= vector_extraction(grbl_out)
    leftside = vector[0]
grbl.write('g90\n'.encode('utf-8'))
grbl_out = grbl.readline()    
grbl.write('g0 x0\n'.encode('utf-8'))
grbl_out = grbl.readline()
print('return to center: ', grbl_out.decode('utf-8'))
grbl.write('g91\n'.encode('utf-8'))
grbl_out = grbl.readline()
grbl.write(('g38.2 x-' + str(approxCircleDia/2) + ' f' + str(probeFeed)+'\n').encode('utf-8'))
grbl_out = grbl.readline()
print('left probe: ',grbl_out.decode('utf-8'))
grbl_out = grbl.readline()
grbl_out = grbl_out.strip().decode('utf-8') 
print('position :', grbl_out)
if grbl_out[0:4] == '[PRB':  
    vector= vector_extraction(grbl_out)
    rightside = vector[0]
    
Xdistance = leftside - rightside
grbl.write('g91\n'.encode('utf-8'))
grbl_out = grbl.readline()
print('increment mode: ',grbl_out.decode('utf-8'))
grbl.write(('g0 x' + str(Xdistance/2)+'\n').encode('utf-8'))
grbl_out = grbl.readline()
print('go to x center: ',grbl_out.decode('utf-8'))
grbl.write('g10 p1 l20 x0\n'.encode('utf-8')) 
grbl_out = grbl.readline().strip().decode('utf-8')
#a=input("Si se ha completado correctamente, pulse enter.")

#BUSCAMOS EL CENTRO EN Z
           
grbl.write(('g38.2 z'+ str(approxCircleDia/2) + ' f' + str(probeFeed)+'\n').encode('utf-8'))
grbl_out = grbl.readline()
print('top probe: ',grbl_out.decode('utf-8'))
grbl_out = grbl.readline()
   
grbl.write('g91\n'.encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
grbl.write(('g0 z-19\n').encode('utf-8'))
grbl_out = grbl.readline()

print('desplazamiento al centro: ',grbl_out.strip().decode('utf-8'))
grbl.write(('g10 p1 l20 z0\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('centro encontrado: ',grbl_out)    

a=input("If everything is correct, press enter to continue.")

#BUSCAMOS EL CENTRO EN Y

grbl.write(('g91\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')

grbl.write(('g0 y6.5\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print(grbl_out)
grbl.write(('g0 x'+str(24+2.5)+'\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print(grbl_out)
grbl.write(('g38.2 y-10 f'+ str(int(probeFeed/2))+'\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('probe y: ',grbl_out)
grbl_out = grbl.readline().strip().decode('utf-8')
print('position: ',grbl_out)
grbl_out = grbl.readline().strip().decode('utf-8')
print('position: ',grbl_out)

grbl.write(('g10 p1 l20 y0\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('1?',grbl_out)
grbl.write(('g91\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('1.2?',grbl_out)

grbl.write(('g01 y2 f300\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('2?',grbl_out)

a = input("If everything is correct, press enter to continue.")

grbl.write(('g90\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('3?',grbl_out)
grbl.write(('g0 x0y0z0\n').encode('utf-8'))
grbl_out = grbl.readline().strip().decode('utf-8')
print('4?',grbl_out)
time.sleep(1)

a = input("If everything is correct, press enter to continue.")

grbl.write(('g0 X-3.53Y-33.15Z6.23\n').encode('utf-8')) #place the sensitive spot in the center 
grbl_out = grbl.readline

grbl.write('g10 p1 l20 x0 y0 z0\n'.encode('utf-8'))
grbl_out = grbl.readline()

a=input('Press any key when done ->')

grbl.close()  

alarm quit:  [MSG:Caution: Unlocked]

right probe:  ok

position:  [PRB:16.886,0.000,0.100:1]
return to center:  ok

left probe:  ok

position : [PRB:-12.506,0.000,0.100:1]
increment mode:  ok

go to x center:  ok

top probe:  ok

desplazamiento al centro:  ok
centro encontrado:  ok

ok
ok
probe y:  ok
position:  [PRB:28.433,2.993,-11.035:1]
position:  ok
1? ok
1.2? ok
2? ok

3? ok
4? ok

Press any key when done ->


# Magnetic measurements in a cylinder
First check that the diameter and positions along the y axis will not surpass the limits of the XYZ positioner.

In [13]:
start = time.time()

#INTRODUCE THE VALUES FOR THE MEASUREMENT

diameter = 8            #diameter of the circumference
yvector = [0, 0,-10.0,-30.45,-38.59,-46.75,-50.75,-58.9,-67.05,-71.05,-79.2,-87.35,-91.35,-99.5,-107.65,-111.65,-119.80000000000001,-127.94999999999999,-131.95,-140.1,-148.25,-152.25,-160.4,-168.55,-181.0]
yvector_e = yvector[0:20]

#Those are the positions that correspond to the begining mid and end planes in the subcoils. 
#Also starting and ending points of the steel casing. It performs two circles at y=0 because sometimes 
#the first measurements are erratic. Beware! Those are the points for the whole solenoid, depending on the placement 
#of the machine, maybe not all are possible to make. Previously check the y movement limit.

precision = 1             #distance between points in the grid
FeedRate = 200            #movement of the motor between points
num = int(300/100)        #Arduino sends the average of 100 measurements. 
                          #Num gives how many hundreds of measurements are wanted.

last_movement = [0,0,0]


# Open serial port for arduino 

arduino = serial.Serial('COM3',115200)
for i in range(5):
    arduino_out = arduino.readline()
    #print(arduino_out)
nom = input("Introduce a name for the file (without .txt):")
fichero = open(nom+".txt","w") 

#DO IT: Find the centre of the cylinder with the code provided.



try:
    #Open serial port for the CNC
    grbl = serial.Serial('COM4',115200) 

    # Wake up grbl
    grbl.write('\r\n\r\n'.encode('utf-8'))
    time.sleep(2.0)   # Wait for grbl to initialize
    grbl.flushInput()  # Flush startup text in serial input
    grbl.write('$x\n'.encode('utf-8'))
    time.sleep(1)
    grbl_out = grbl.readline() # Wait for grbl response with carriage return
    print( ' : ' + grbl_out.strip().decode('utf-8'))
    grbl.write('g10 p1 l20 x0 y0 z0\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g21\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g90\n'.encode('utf-8'))
    out = grbl.readline()
    # Stream g-code to grbl
    
    
    #Generate the coordinates for all the poins in a cylinder
    MyCylinder = cylinder_generator(diameter,yvector_e,precision) 
     


    for section in MyCylinder:             #go through all the vectors and send their direction to the grbl
        for vector in section:
            gcode_command = ('G01 x' + str(vector[0]) + 'y' + str(vector[1]) + 'z' + str(vector[2]) + 'f'+str(FeedRate)+'\n')
            #print ('Sending: ' + gcode_command)
            grbl.write(gcode_command.encode('utf-8'))
            grbl_out = grbl.readline()
            #print( ' : ' + grbl_out.strip().decode('utf-8'))
            mag_m = np.array([0.0,0.0,0.0])
            rep = num
            #now we compute the approximate time it takes to make the following movement
            current_movement = np.array([vector[0],vector[1],vector[2]])
            distance = (last_movement - current_movement)
            mod_distance = (distance[0]**2+distance[1]**2+distance[2]**2)**0.5
            movement_time = mod_distance / FeedRate * 60
            if (mod_distance>3):           #if the movement is one of the long ones, when changin from section, 
                                           #we add a plus to make certain it does not start measuring while moving
                movement_time+=4
            time.sleep(movement_time + 0.1)
            for i in range(num):
                try:
                    mag_reading = arduino.readline().decode('utf-8').rstrip()
                    mag_v = mag_reading.split(',')
                #print(mag_v)

                    mag_v = np.array([float(mag_v[0]),float(mag_v[1]),float(mag_v[2])])
                    mag_m += mag_v
                except:
                    rep -= 1
                time.sleep(0.0005)
            if ((num-rep) != 0):
                print('No se han tomado',num-rep,'medidas.')
            mag_m = mag_m/rep
            fichero.write(str(round(1000*vector[0]/1000))+', '+str(round(1000*vector[1]/1000))+', '+str(round(1000*vector[2]/1000))+', '+str(round(1000*mag_m[0])/1000)+', '+str(round(1000*mag_m[1])/1000)+', '+str(round(1000*mag_m[2])/1000)+'\n')
            last_movement = np.array([vector[0],vector[1],vector[2]])

    
finally:
    # Close file and serial port
    grbl.write('g0 x0y0z0\n'.encode('utf-8'))
    grbl.readline()
    finish = time.time()
    print("ha tardado: ",finish-start)
    input("Press <Enter> to exit and disable grbl.")
    fichero.close()
    grbl.close()
    arduino.close()

Introduce un nombre para el fichero:magnetic_cylinder2_day2
 : [MSG:Caution: Unlocked]
ha tardado:  1427.1150498390198
Press <Enter> to exit and disable grbl.


# Magnetic measurements along the axis
This piece of code performs measurements along a line with x and z constant. Move the sensor to the desired x and z coordinates and change, if needed, the distance to travel and the distance between measurements.

In [46]:
FeedRate = 200
distance = 150
precision = 1



# Open serial port for arduino 
arduino = serial.Serial('COM3',115200)
for i in range(5):
    arduino_out = arduino.readline()
    #print(arduino_out)
nom=input("Introduce un nombre para el fichero:")
fichero=open(nom + ".txt","w") 
num = 3


try:
    grbl = serial.Serial('COM4',115200) #Open serial port for the CNC

    # Wake up grbl
    grbl.write('\r\n\r\n'.encode('utf-8'))
    time.sleep(2.0)   # Wait for grbl to initialize
    grbl.flushInput()  # Flush startup text in serial input
    grbl.write('$x\n'.encode('utf-8'))
    time.sleep(1)
    grbl_out = grbl.readline() # Wait for grbl response with carriage return
    print( ' : ' + grbl_out.strip().decode('utf-8'))
    grbl.write('g10 p1 l20 x0 y0 z0\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g21\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g91\n'.encode('utf-8'))
    out = grbl.readline()
    


    grbl.write(('g0 y5\n').encode('utf-8'))
    out = grbl.readline()
    #print(section)
    for i in range (int(distance/precision)+1):
        gcode_command = ('G0 y-' + str(precision) + '\n')
        #print ('Sending: ' + gcode_command)
        grbl.write(gcode_command.encode('utf-8'))
        grbl_out = grbl.readline()
        #print( ' : ' + grbl_out.strip().decode('utf-8'))
        mag_m = np.array([0.0,0.0,0.0])
        rep = num
        time.sleep(0.1)
        for j in range(num):
            try:
                mag_reading = arduino.readline().decode('utf-8').rstrip()
                mag_v = mag_reading.split(',')
            #print(mag_v)
            
                mag_v = np.array([float(mag_v[0]),float(mag_v[1]),float(mag_v[2])])
                mag_m += mag_v
            except:
                rep -= 1
            if ((num-rep) != 0):
                print(num-rep,'measurements have not been taken.')       
        mag_m = mag_m/rep
        fichero.write(str(-distance/2+i*precision)+ ', ' +str(round(1000*mag_m[0])/1000)+', '+str(round(1000*mag_m[1])/1000)+', '+str(round(1000*mag_m[2])/1000)+'\n')
        
    
finally:
    # Close file and serial port
    grbl.write('g90\n'.encode('utf-8'))
    out = grbl.readline()
    grbl.write('g0 y0\n'.encode('utf-8'))
    out = grbl.readline()
    grbl.write('g04 p2\n'.encode('utf-8'))
    out = grbl.readline()
    input("  Press <Enter> to exit and disable grbl.")
    fichero.close()
    grbl.close()
    arduino.close()
    
    
    

Introduce un nombre para el fichero:magnetic_axis7_z6
 : [MSG:Caution: Unlocked]
No se han tomado 1 medidas.
No se han tomado 1 medidas.
No se han tomado 1 medidas.
  Press <Enter> to exit and disable grbl.


# Crosses
This piece of code performs cross-like measurements in different points of the y axis specified in yvector. You can change the height and width of the cross, the distance between measuremens, the FeedRate of the motors and the number of measurements per point(x100).

In [10]:
start = time.time()

#INTRODUCE THE VALUES FOR THE MEASUREMENT

height = 12
width = 12
yvector = [0, 0,-10.0,-30.450000000000003,-38.599999999999994,-46.75,-50.75,-58.900000000000006,-67.05,-71.05,-79.2,-87.35,-91.35,-99.5,-107.65,-111.65,-119.80000000000001,-127.94999999999999,-131.95,-140.1,-148.25,-152.25,-160.4,-168.55,-181.0]
precision = 1
FeedRate = 200
num = int(300/100)


last_movement = [0,0,0]


# Open serial port for arduino 

arduino = serial.Serial('COM3',115200)
for i in range(5):
    arduino_out = arduino.readline()
    #print(arduino_out)
nom = input("Introduce un nombre para el fichero:")
fichero = open(nom+".txt","w") 

#Find the centre of the cylinder

#center_probe()


try:
    grbl = serial.Serial('COM4',115200) #Open serial port for the CNC

    # Wake up grbl
    grbl.write('\r\n\r\n'.encode('utf-8'))
    time.sleep(2.0)   # Wait for grbl to initialize
    grbl.flushInput()  # Flush startup text in serial input
    grbl.write('$x\n'.encode('utf-8'))
    time.sleep(1)
    grbl_out = grbl.readline() # Wait for grbl response with carriage return
    print( ' : ' + grbl_out.strip().decode('utf-8'))
    grbl.write('g10 p1 l20 x0 y0 z0\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g21\n'.encode('utf-8'))
    grbl.readline()
    grbl.write('g90\n'.encode('utf-8'))
    out = grbl.readline()
    # Stream g-code to grbl

    
  
    for ycord in yvector[0:20]:
        movement_time = abs(ycord)/FeedRate*60
        grbl.write(('g01 y'+str(ycord)+'F'+str(FeedRate)+'\n').encode('utf-8'))
        grbl.readline()
        time.sleep(movement_time)
        grbl.write('g91\n'.encode('utf-8'))
        grbl.readline()
        grbl.write(('g01 z-'+str(height/2)+'F'+str(FeedRate)+'\n').encode('utf-8'))
        grbl.readline()
        movement_time = height/2/FeedRate*60
        time.sleep(movement_time+0.7)
        mag_m = np.array([0.0,0.0,0.0])
        rep = num
        movement_time=precision/FeedRate*60
        for i in range (int(height/precision)):
            grbl.write(('g01 z'+str(precision)+'F'+str(FeedRate)+'\n').encode('utf-8'))
            grbl.readline()
            time.sleep(movement_time + 0.1)
            mag_m = np.array([0.0,0.0,0.0])
            for i in range(num):
                try:
                    mag_reading = arduino.readline().decode('utf-8').rstrip()
                    mag_v = mag_reading.split(',')
                #print(mag_v)

                    mag_v = np.array([float(mag_v[0]),float(mag_v[1]),float(mag_v[2])])
                    mag_m += mag_v
                except:
                    rep -= 1
                time.sleep(0.0005)
            if ((num-rep) != 0):
                print('No se han tomado',num-rep,'medidas.')
            mag_m = mag_m/rep
            fichero.write(str(round(1000*0/1000))+', '+str(round(1000*ycord/1000))+', '+str(round(1000*(-height/2+i*precision)/1000))+', '+str(round(1000*mag_m[0])/1000)+', '+str(round(1000*mag_m[1])/1000)+', '+str(round(1000*mag_m[2])/1000)+'\n')
        grbl.write('g90\n'.encode('utf-8'))
        grbl.readline()
        grbl.write('g0 z0\n'.encode('utf-8'))
        grbl.write('g91\n'.encode('utf-8'))
        grbl.readline()
        grbl.write(('g01 x-'+str(width/2)+'F'+str(FeedRate)+'\n').encode('utf-8'))
        grbl.readline()
        movement_time = height/2/FeedRate*60
        time.sleep(movement_time+0.7)
        movement_time = precision/FeedRate*60
        rep = num
        for i in range (int(width/precision)):
            grbl.write(('g01 x'+str(precision)+'F'+str(FeedRate)+'\n').encode('utf-8'))
            grbl.readline()
            time.sleep(movement_time + 0.1)
            mag_m = np.array([0.0,0.0,0.0])
            for i in range(num):
                try:
                    mag_reading = arduino.readline().decode('utf-8').rstrip()
                    mag_v = mag_reading.split(',')
                #print(mag_v)

                    mag_v = np.array([float(mag_v[0]),float(mag_v[1]),float(mag_v[2])])
                    mag_m += mag_v
                except:
                    rep -= 1
                time.sleep(0.0005)
            if ((num-rep) != 0):
                print('No se han tomado',num-rep,'medidas.')
            mag_m = mag_m/rep
            fichero.write(str(round(1000*(-width/2+i*precision)/1000))+', '+str(round(1000*ycord/1000))+', '+str(round(1000*0/1000))+', '+str(round(1000*mag_m[0])/1000)+', '+str(round(1000*mag_m[1])/1000)+', '+str(round(1000*mag_m[2])/1000)+'\n')
        
        grbl.write('g90\n'.encode('utf-8'))
        grbl.readline()
        grbl.write('g0 x0\n'.encode('utf-8'))
               
finally:
    # Close file and serial port
    grbl.write('g0 x0y0z0\n'.encode('utf-8'))
    grbl.readline()
    finish = time.time()
    print("ha tardado: ",finish-start)
    input("Press <Enter> to exit and disable grbl.")
    fichero.close()
    grbl.close()
    arduino.close()

Introduce un nombre para el fichero:magnetic_cross_1
 : [MSG:Caution: Unlocked]
ha tardado:  843.7507371902466
Press <Enter> to exit and disable grbl.
